In [2]:
import os
from modulus.sym.hydra import to_absolute_path
from csv_rw import csv_to_dict
import csv
import numpy as np
import glob
from multiprocessing import Process
from fwdFacingStep import ffs, param_ranges, Re, Ho, Lo


In [30]:
def readFile(file_path):
    with open(file_path, "r") as datafile:
        data = []
        reader = csv.reader(datafile, delimiter=",")
        for row in reader:
            columns = [row[1]]
            data.append(columns)
        last_row = float(data[-1][0])
        return last_row

In [2]:
# Read and Compare Delta Cp

from os import listdir

ansysFilePath="./ansysDPsValidation.csv"
resultsFilePath="./results.csv"
outputsPath="./outputs/fwdFacingStep"

# models = ["old_dataPlusPhysics3600@300k", "data3600PlusPhysicsLambda05@300k", "data3600PlusPhysicsLambda1@300k", "old_physicsOnly@500k", "physicsOnly@500k"]

dirSkip = [".hydra", "init"]

models = listdir(outputsPath)
models.sort()

models = ["physicsOnly@500k", "data1800PlusPhysicsLambda01@500k", "data1800PlusPhysicsLambda1@500k"]


diffSumSkip = ["DP 5","DP 36","DP 79","DP 86"]


with open(ansysFilePath, "r") as ansysFile,  open(resultsFilePath, "w") as resultsFile:
        reader = csv.reader(ansysFile, delimiter=",")
        writer = csv.writer(resultsFile, delimiter=",")
        diffDict= {}
        for model in models:
                diffDict[model] = 0
                # print(type(diffDict[model]))
        for i, row in enumerate(reader):
                if i == 0:
                        # add columns for difference modulus vs ansys
                        row.insert(5, "Diff Delta Cp [%]")
                        # row.insert(7, "Diff Delta Cptot [%]")
                        writer.writerow(row)
                else:
                        # add ansys results to table
                        row.insert(5, "-")
                        # row.insert(7, "-")
                        writer.writerow(row)
                        for model in models:
                                print(model)
                                if model in dirSkip:
                                        print("skipping ", model)
                                        continue
                                
                                monitorFilePath = os.path.join(outputsPath, model, "monitors")
                                # read pressure monitors
                                filePattern="downstreamPressure_" + row[0].replace(" ","") + "_*"
                                monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                DSP=readFile(monitorFile[0])
                                # filePattern="downstreamPressureTot_" + row[0].replace(" ","") + "_*"
                                # monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                # DSPtot=readFile(monitorFile[0])
                                filePattern="upstreamPressure_" + row[0].replace(" ","") + "_*"
                                monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                USP=readFile(monitorFile[0])
                                # filePattern="upstreamPressureTot_" + row[0].replace(" ","") + "_*"
                                # monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                # USPtot=readFile(monitorFile[0])
                                
                                # calculate modulus pressure drop and difference modulus vs ansys
                                deltaCp=2*(USP-DSP)
                                # deltaCptot=2*(USPtot-DSPtot)
                                diffDeltaCp=100*(deltaCp-float(row[4]))/float(row[4])
                                # diffDeltaCptot=100*(deltaCptot-float(row[6]))/float(row[6])
                                
                                if row[0] not in diffSumSkip:
                                        print("ns", row[0])
                                        diffDict[model] += abs(diffDeltaCp)
                                
                                # add modulus results to table
                                modRow=[row[0] + " " + model] + row[1:4] + [deltaCp] + [diffDeltaCp] + [DSP] + [USP]
                                # modRow=[row[0] + " " + model] + row[1:4] + [deltaCp] + [diffDeltaCp] + [deltaCptot] + [diffDeltaCptot] + [DSP] + [DSPtot] + [USP] + [USPtot]
                                writer.writerow(modRow)
        # Add deviation sum
        for model in models:
                deviationRow = ["Sum " + model] + ["-"] + ["-"] + ["-"] + ["-"] + [diffDict[model]] + ["-"] + ["-"]
                writer.writerow(deviationRow)
                
                                
                                


physicsOnly@500k


NameError: name 'os' is not defined

In [1]:
# Read Optimization Results

import csv
from os import listdir
import numpy as np

resultsFilePath="./resultsOpt.csv"

dirSkip = [".hydra", "init"]

optResultsPath = "./optimizationResults/"

# models = ["physicsOnly@500k", "data1800PlusPhysicsLambda01@300k"]
models = ["physicsOnly@500k", "data1800PlusPhysicsLambda01@500k"]
# models = listdir(optResultsPath)
# models.sort()

with open(resultsFilePath, "w") as resultsFile:
    writer = csv.writer(resultsFile, delimiter=",")
    
    firstRow = ["model", "Re", "Lo", "Ho", "delta Cp", "delta Cp Ref"]
    writer.writerow(firstRow)


    for model in models:
        if model in dirSkip:
            print("skipping ", model)
            continue
            
        optPath = optResultsPath + model
        for reNr in range (100, 1100, 100):

            F = np.load(optPath + "/optResultsF" + str(reNr) + ".npy")
            X = np.load(optPath + "/optResultsX" + str(reNr) + ".npy")
            popF = np.load(optPath + "/popF" + str(reNr) + ".npy")
            popX = np.load(optPath + "/popX" + str(reNr) + ".npy") 
            # print("Re " + str(re) + " F: ", F)
            # print("Re " + str(reNr) + ": Lo = " + str(X[0]), " Ho = " + str(X[1]))
            
            row = [model]
            row += [reNr, X[0], X[1], F[0]]
            
            if reNr == 100:
                latexStr = model + " & " + str(reNr)
            else:
                latexStr = " " + " & " + str(reNr)
            
            for value in row[2:]:
                # print(value)
                valueF = round(float(value), 4)
                latexStr += " & " + "%.4f" % valueF
            latexStr += " & - "
            latexStr += " \\\\"
            

            
            writer.writerow(row)
            print(latexStr)
        print("\\hline")


physicsOnly@500k & 100 & 0.2117 & 0.1000 & 20.1844 & -  \\
  & 200 & 0.2000 & 0.1000 & 14.6800 & -  \\
  & 300 & 0.2000 & 0.3328 & 12.4632 & -  \\
  & 400 & 0.2000 & 0.3361 & 11.2295 & -  \\
  & 500 & 0.2000 & 0.3370 & 10.4113 & -  \\
  & 600 & 0.2000 & 0.3390 & 9.8743 & -  \\
  & 700 & 0.2000 & 0.3407 & 9.4838 & -  \\
  & 800 & 0.2000 & 0.3426 & 9.1810 & -  \\
  & 900 & 0.2000 & 0.3433 & 8.9780 & -  \\
  & 1000 & 0.2000 & 0.3436 & 8.9407 & -  \\
\hline


FileNotFoundError: [Errno 2] No such file or directory: './optimizationResults/data1800PlusPhysicsLambda01@500k/optResultsF100.npy'

['pressureDataPlusPhysicsLambda01@500k', 'data3600PlusPhysicsLambda01@500k', 'data1800PlusPhysicsLambda01@300k', 'pressureDataPlusPhysicsLambda05@300k', '.hydra', 'physicsOnly@100k', 'data3600PlusPhysicsLambda01@300k', 'data3600PlusPhysicsLambda05@300k', 'data3600PlusPhysicsLambda1@300k', 'data3600PlusPhysicsLambda05@500k', 'pressureDataPlusPhysicsLambda05@100k', 'data3600PlusPhysicsLambda01@100k', 'pressureDataPlusPhysicsLambda01@100k', 'data3600PlusPhysicsLambda05@100k', 'physicsOnly@300k', 'physicsOnly@500k', 'data3600PlusPhysicsLambda1@100k', 'data1800PlusPhysicsLambda01@500k', 'data3600PlusPhysicsLambda1@500k', 'data1800PlusPhysicsLambda01@100k', 'pressureDataPlusPhysicsLambda01@300k']


('hej', 2)